In [5]:
import pandas as pd
import folium
import numpy as np

In [6]:
df_user = pd.read_csv('./data/OrderAprtoJun.csv')

F:\acconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (17,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def geospatial_plot(df_kitchen, name='kitchen1', sample_frac=1):
    df_loc = df_kitchen[['user_longitude', 'user_latitude', 'distance']].rename(columns={'user_longitude':'longitude', 'user_latitude':'latitude'})
    # subsampling
    sample_df = df_loc.sample(frac = sample_frac)
    print('subsampling with {} of the whole data, {} properties in total'.format(sample_frac, len(sample_df)))

    # creating base map
    center = (sample_df.latitude.mean(), sample_df.longitude.mean())
    map_age = folium.Map(location=[center[0], center[1]], zoom_start=9, prefer_canvas=True)

    locations = list(zip(sample_df.latitude, sample_df.longitude))
    popups = ['lon:{}<br>lat:{}'.format(lon, lat) for (lat, lon) in locations]
    
    icon_create_function = """\
    function(cluster) {
        return L.divIcon({
        html: '<b>' + cluster.getChildCount() + '</b>',
        className: 'marker-cluster marker-cluster-large',
        iconSize: new L.Point(20, 20)
        });
    }"""
    
    m = folium.Map(
        location=[sample_df.latitude.mean(), sample_df.longitude.mean()],
        tiles='Cartodb Positron',
        zoom_start=6,
        prefer_canvas=True
    )

    marker_cluster = MarkerCluster(
        locations=locations, popups=popups,
        name='1000 clustered icons',
        overlay=True,
        control=True,
        icon_create_function=icon_create_function
    )

    marker_cluster.add_to(m)

    folium.LayerControl().add_to(m)
    m.save(name + '.html')

In [8]:
df_loc = df_user[['user_longitude', 'user_latitude']].rename(columns={'user_longitude':'longitude', 'user_latitude':'latitude'})

In [9]:
len(df_loc)

1367885

## Now, we want to see the most popular kitchens

In [10]:
df_user.groupby('kitchen_id')['kitchen_id'].count().sort_values(ascending=False).head(10)

kitchen_id
9575     4294
18170    4229
47767    3801
36221    3527
11758    3438
65788    2741
86535    2707
54053    2615
3740     2572
55323    2571
Name: kitchen_id, dtype: int64

In [12]:
df_kitchen = df_user.loc[df_user['kitchen_id']==9575]

### user location and distance

In [3]:
df_user.head()

,Unnamed: 0,order_no,kitchen_id,user_id,city_id,send_time,send_type,state,state_train,dishes,...,coupon_id,coupon_fee,platform,app_version,update_time,insure,ticket_id,ticket_fee,raw_data,dt
0,0,463044790017929,28366,101296988,110100,2016-05-12 18:30:00.0,1,11,"2,4,6,9,10,11",NaN,...,0,0.0,iOS,2.7.0,2016-05-12 17:55:19.0,1,0,0,NaN,NaN
1,1,463044790017933,17873,101057772,110100,2016-05-12 18:30:00.0,1,11,"2,4,6,8,11",NaN,...,0,0.0,iOS,2.7.0,2016-05-12 18:06:29.0,1,0,0,NaN,NaN
2,2,463044784017918,130952,100789540,110100,2016-05-12 18:30:00.0,1,11,"2,4,6,9,10,11",NaN,...,17155317,8.0,iOS,2.7.0,2016-05-12 17:32:49.0,1,0,0,NaN,NaN
3,3,463044770017889,151016,101057084,440300,2016-05-12 18:30:00.0,1,11,"2,4,6,9,10,11",NaN,...,17164655,8.0,Android,2.7.1,2016-05-12 17:55:12.0,1,0,0,NaN,NaN
4,4,463044778017902,148024,101370266,110100,2016-05-12 18:30:00.0,1,11,"2,4,6,9,10,11",NaN,...,0,0.0,Android,2.7.1,2016-05-12 17:58:44.0,1,0,0,NaN,NaN


In [25]:
kitchen_distance = df_user.groupby('kitchen_id')['distance'].median()

In [9]:
df_price = pd.read_csv('./data/dish_info.csv')

In [10]:
df_price = df_price[pd.notnull(df_price['name'])]
df_price = df_price[df_price['price'] <= 100]

In [26]:
kitchen_price = df_price.groupby('kitchen_id')['price'].median()

In [27]:
df_pd = pd.concat([kitchen_distance, kitchen_price], axis=1).dropna()

In [28]:
df_pd['distance'].corr(df_pd['price'])

7.69912823759963e-05

In [46]:
df_price.groupby('kitchen_id')['price'].mean()[df_price.groupby('kitchen_id')['price'].mean().index == 18170]

kitchen_id
18170    24.011111
Name: price, dtype: float64

In [47]:
df_user.groupby('kitchen_id')['distance'].median()[df_user.groupby('kitchen_id')['distance'].median().index == 18170]

kitchen_id
18170    1214.0
Name: distance, dtype: float64

In [49]:
df_user.groupby('kitchen_id')['distance'].median().describe()

count      5523.000000
mean       1404.887380
std       11647.534718
min           0.000000
25%         974.000000
50%        1280.000000
75%        1580.000000
max      865739.000000
Name: distance, dtype: float64

In [ ]:
df_price.groupby('')

In [78]:
df_price.groupby('kitchen_id').price.count()

kitchen_id
13          1
30         18
55        209
57          1
62          6
66        187
158        29
166        23
182        38
210         2
226         6
229        38
234        52
314        18
400        82
415        45
421       310
459        25
471        15
472        34
473        52
499        13
524        47
594        31
626        16
698        88
708        40
852        57
879         4
946         5
         ... 
172884      3
172904      3
172906      8
172908      3
172915      7
172954      3
172976      3
172983      3
172993      3
173006      3
173075      3
173114      3
173128      3
173158      3
173169      3
173175      3
173218      3
173266      3
173302      3
173371      3
173375      3
173391      3
173407      3
173418      3
173522      3
173585      3
173660      3
173813      3
173822      3
174091     10
Name: price, Length: 16539, dtype: int64

In [84]:
df_price.loc[df_price['kitchen_id'] == 55]

,index,dish_id,kitchen_id,name,price,volume,stock,tomorrow_stock,daily_stock,is_recommend,...,tmr_only,dish_feature,set_meal,sequence,source,materials,is_new,cooking_time,net_price,sequence_in_catalogue
3903,5628,298637,55,生爆盐煎肉,26,0,10,99,10,0,...,0,"湘菜,北京菜,家常菜",0.0,1.0,1.0,NaN,0.0,0.0,26.0,29.0
8700,12631,305640,55,土豆片小炒肉,23,0,10,99,10,0,...,0,湘菜*北京菜*家常菜,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
8706,12637,305646,55,榨菜肉丝,21,0,10,99,10,0,...,0,湘菜*北京菜*家常菜,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
10500,15295,308304,55,娃娃菜炒肉,23,0,15,99,15,0,...,0,湘菜*北京菜*家常菜,0.0,1.0,1.0,NaN,0.0,0.0,23.0,30.0
10638,15516,308525,55,土豆片小炒肉,31,0,15,99,15,0,...,0,"北京菜,湘菜,家常菜",0.0,8.0,1.0,NaN,0.0,5.0,28.0,0.0
10648,15526,308535,55,榨菜炒肉丝,20,0,15,99,15,0,...,0,湘菜*粤菜*北京菜,0.0,1.0,1.0,NaN,0.0,0.0,20.0,31.0
10660,15546,308555,55,自制排骨,31,0,5,99,5,0,...,0,"湘菜,北京菜,,家常菜",0.0,1.0,1.0,NaN,0.0,0.0,31.0,32.0
11134,16292,309301,55,尖椒青椒炒鸡蛋,18,0,15,99,15,0,...,0,"湘菜,北京菜,,家常菜",0.0,1.0,1.0,NaN,0.0,0.0,18.0,33.0
12539,18507,311516,55,红烧肉炖土豆,30,0,9,99,10,1,...,0,"湘菜,北京菜,家常菜",0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
12546,18514,311523,55,红烧肉炖胡萝卜,28,0,10,99,10,0,...,0,"湘菜,山西菜,,家常菜",0.0,1.0,1.0,NaN,0.0,0.0,28.0,34.0


In [86]:
df_price.loc[df_price['name'] == '米饭']['eat_num']

154         0
168        41
181         1
186       325
459         3
564        24
586         1
647        32
822        32
946         0
993        46
1010        1
1063        0
1131       27
1134        4
1269       23
1308       13
1320       18
1381       12
1430        1
1454       47
1470       15
1515        1
1536       12
1541       17
1551        1
1975        0
2461        0
2471      258
2545        0
         ... 
380993     21
381042      4
381167      1
381208    256
381269      0
381303    129
381383      0
381544    128
381682      6
381735     13
381834      0
381836     49
381885      0
381960      0
381981     11
382035      2
382112      8
382151     28
382409     18
382413     21
382428      1
382430      1
382535      0
382583      0
382589      3
382711     15
382746      4
382757     62
382842      2
382851      8
Name: eat_num, Length: 3882, dtype: int64

In [88]:
df_price.loc[df_price['kitchen_id'] == 55][['price','eat_num']]

,price,eat_num
3903,26,2
8700,23,0
8706,21,0
10500,23,0
10638,31,71
10648,20,0
10660,31,0
11134,18,1
12539,30,2
12546,28,1


In [90]:
df_price[df_price['price'] < 10]

,index,dish_id,kitchen_id,name,price,volume,stock,tomorrow_stock,daily_stock,is_recommend,...,tmr_only,dish_feature,set_meal,sequence,source,materials,is_new,cooking_time,net_price,sequence_in_catalogue
56,68,293073,86279,东北长粒香米饭,3,0,5,99,5,0,...,0,五谷杂粮,0.0,1.0,1.0,NaN,0.0,0.0,3.0,3.0
66,86,293091,84075,玉米饼,9,0,5,99,5,0,...,0,家常饼,0.0,1.0,1.0,NaN,0.0,0.0,9.0,5.0
139,190,293195,84075,小米粥,4,0,10,99,10,0,...,0,粥,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
144,204,293209,79002,荷包蛋鸡蛋,2,0,40,99,40,0,...,0,江浙菜*家常菜,0.0,17.0,1.0,NaN,0.0,0.0,2.0,4.0
152,216,293221,77320,秘制茶叶蛋,2,0,10,99,10,0,...,0,湘菜*家常菜,0.0,1.0,1.0,NaN,0.0,0.0,2.0,1.0
154,218,293223,68756,米饭,2,0,20,99,20,0,...,0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,2.0,1.0
168,235,293240,71257,米饭,2,0,10,99,10,0,...,0,江浙菜,0.0,1.0,1.0,NaN,0.0,0.0,2.0,3.0
181,264,293269,77158,米饭,2,0,10,99,10,0,...,0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
183,272,293277,19824,大米饭,2,0,10,99,10,0,...,0,五谷杂粮,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
186,276,293281,66703,米饭,2,0,20,99,20,0,...,0,东北菜,0.0,1.0,1.0,NaN,0.0,5.0,2.0,0.0


In [96]:
df_price[df_price['name'].str.contains('米饭')][df_price[df_price['name'].str.contains('米饭')].price > 10]

,index,dish_id,kitchen_id,name,price,volume,stock,tomorrow_stock,daily_stock,is_recommend,...,tmr_only,dish_feature,set_meal,sequence,source,materials,is_new,cooking_time,net_price,sequence_in_catalogue
617,972,293981,67091,鸡蛋葱花炒米饭,12,0,5,99,5,0,...,0,北京菜,0.0,1.0,1.0,NaN,0.0,0.0,12.0,2.0
680,1071,294080,38793,黄焖鸡米饭,22,0,2,99,2,0,...,0,北京菜,0.0,1.0,1.0,NaN,0.0,0.0,21.0,32.0
694,1089,294098,83013,麻辣豆腐送米饭,16,0,5,99,5,1,...,0,鲁菜,0.0,1.0,1.0,NaN,0.0,0.0,14.0,4.0
857,1298,294307,64898,黄焖鸡米饭+辣味,16,0,99,99,99,0,...,0,家常菜,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
1028,1515,294524,93620,酱香茄子送米饭,15,0,3,99,3,0,...,0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,14.0,1.0
1048,1544,294553,98336,卤肉饭（配青菜卤蛋米饭）,28,0,20,99,20,0,...,0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,27.0,1.0
1448,2139,295148,44298,鸡蛋炒米饭,12,0,10,99,10,0,...,0,NaN,0.0,17.0,1.0,NaN,0.0,0.0,12.0,4.0
1800,2658,295667,93620,酸辣土豆丝送米饭,15,0,5,99,5,0,...,0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,14.0,2.0
2271,3295,296304,79649,家香黄焖鸡米饭,22,0,3,99,3,0,...,0,家常菜*炖品,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0
2581,3737,296746,32787,鸡蛋炒米饭,16,0,5,99,5,0,...,1,NaN,0.0,6.0,1.0,NaN,0.0,0.0,16.0,15.0
